In [1]:
import os
import pickle
import torch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Tuple, List, Dict

from cl_explain.metrics.ablation import compute_auc

In [2]:
RESULT_PATH = "/projects/leelab/cl-explainability/results"
SUPERPIXEL_ATTRIBUTION_METHODS = ["kernel_shap"]
SEED_LIST = [123, 456, 789, 42, 91]

In [3]:
def get_eval_filename(
    different_classes: bool,
    comprehensive: bool,
    corpus_size: int,
    explanation_name: str,
    foil_size: int,
    explicand_size: int,
    attribution_name: str,
    superpixel_dim: int,
    removal: str,
    blur_strength: float,
    eval_superpixel_dim: int,
    eval_foil_size: int,
    take_attribution_abs: bool,
) -> str:
    """Get eval filename."""
    if different_classes:
        eval_filename = "diff_class"
    else:
        eval_filename = "same_class"
    if comprehensive:
        eval_filename += "_comprehensive"
        
    eval_filename += "_eval_results"
    eval_filename += f"_explicand_size={explicand_size}"
    if "corpus" in explanation_name:
        eval_filename += f"_corpus_size={corpus_size}"
    if "contrastive" in explanation_name:
        eval_filename += f"_foil_size={foil_size}"
    if attribution_name in SUPERPIXEL_ATTRIBUTION_METHODS:
        eval_filename += f"_superpixel_dim={superpixel_dim}"
    eval_filename += f"_removal={removal}"
    if removal == "blurring":
        eval_filename += f"_blur_strength={blur_strength:.1f}"
    eval_filename += f"_eval_superpixel_dim={eval_superpixel_dim}"
    eval_filename += f"_eval_foil_size={eval_foil_size}"
    if take_attribution_abs:
        eval_filename += "_abs"
    eval_filename += ".pkl"
    return eval_filename


def get_mean_curves(outputs, curve_kind) -> Tuple[List[torch.Tensor], int]:
    available_curve_kinds = ["insertion", "deletion"]
    assert curve_kind in available_curve_kinds, (
        f"curve_kind={curve_kind} is not one of {available_curve_kinds}!"
    )
    target_list = [key for key in outputs.keys()]
    eval_name_list = (
        outputs[target_list[0]]["eval_model_names"]
        + outputs[target_list[0]]["eval_measure_names"]
    )
    eval_mean_curve_dict = {}
    for j, eval_name in enumerate(eval_name_list):
        
        curve_list = []
        num_features = None

        for target, output in outputs.items():
            target_curve_list = (
                output[f"model_{curve_kind}_curves"]
                + output[f"measure_{curve_kind}_curves"]
            )
            curve_list.append(target_curve_list[j])
            num_features = output[f"{curve_kind}_num_features"]
        
        curves = torch.cat(curve_list)
        mean_curve = curves.mean(dim=0).cpu()
        eval_mean_curve_dict[eval_name] = mean_curve
        
    return eval_mean_curve_dict, num_features


def get_auc_stats(
    dataset: str,
    encoder: str,
    attribution: str,
    eval_name: str,
    normalize_similarity: bool,
    different_classes: bool,
    comprehensive: bool = False,
    explicand_size: int = 25,
    removal: str = "blurring",
    blur_strength: float = 5.0,
    superpixel_dim: int = 1,
    eval_superpixel_dim: int = 1,
    foil_size: int = 1500,
    corpus_size: int = 100,
    eval_foil_size: int = 1500,
    take_attribution_abs: bool = False,
) -> Dict[str, Dict[str, List]]:
    if attribution == "random_baseline":
        explanation_list = ["self_weighted"]
    else:
        explanation_list = [  # Make sure to order this way.
            "self_weighted",
            "contrastive_self_weighted",
            "corpus",
            "contrastive_corpus",
        ]
        
    insertion_mean_list = []
    insertion_ci_list = []
    deletion_mean_list = []
    deletion_ci_list = []

    for explanation in explanation_list:
        insertion_list = []
        deletion_list = []
        for seed in SEED_LIST:            
            eval_filename = get_eval_filename(
                different_classes=different_classes,
                comprehensive=comprehensive,
                corpus_size=corpus_size,
                explanation_name=explanation,
                foil_size=foil_size,
                explicand_size=explicand_size,
                attribution_name=attribution,
                superpixel_dim=superpixel_dim,
                removal=removal,
                blur_strength=blur_strength,
                eval_superpixel_dim=eval_superpixel_dim,
                eval_foil_size=eval_foil_size,
                take_attribution_abs=take_attribution_abs,
            )

            if normalize_similarity:
                method_name = "normalized"
            else:
                method_name = "unnormalized"
            method_name += f"_{explanation}_{attribution}"
            with open(
                os.path.join(
                    RESULT_PATH,
                    dataset,
                    encoder,
                    method_name,
                    f"{seed}",
                    eval_filename,
                ),
                "rb",
            ) as handle:
                outputs = pickle.load(handle)
            insertion_curve_dict, insertion_num_features = get_mean_curves(
                outputs, "insertion"
            )
            deletion_curve_dict, deletion_num_features = get_mean_curves(
                outputs, "deletion"
            )
            insertion_list.append(
                compute_auc(
                    curve=insertion_curve_dict[eval_name],
                    num_features=insertion_num_features,
                )
            )
            deletion_list.append(
                compute_auc(
                    curve=deletion_curve_dict[eval_name],
                    num_features=deletion_num_features,
                )
            )
        insertion_mean_list.append(np.mean(insertion_list))
        insertion_ci_list.append(1.96 * np.std(insertion_list) / np.sqrt(len(SEED_LIST)))
        deletion_mean_list.append(np.mean(deletion_list))
        deletion_ci_list.append(1.96 * np.std(deletion_list) / np.sqrt(len(SEED_LIST)))
    return {
        "insertion": {"mean": insertion_mean_list, "ci": insertion_ci_list},
        "deletion": {"mean": deletion_mean_list, "ci": deletion_ci_list},
    }


def get_formatted_aucs(
    insertion_direction: str,
    deletion_direction: str,
    bold_best: bool = True,
    **kwargs,
):
    auc_stats = get_auc_stats(**kwargs)
    
    insertion_mean_list = auc_stats["insertion"]["mean"]
    insertion_ci_list = auc_stats["insertion"]["ci"]
    if insertion_direction == "max":
        insertion_best_idx = np.argmax(insertion_mean_list)
    elif insertion_direction == "min":
        insertion_best_idx = np.argmin(insertion_mean_list)
    else:
        raise ValueError(
            f"insertion_direction={insertion_direction} should be max or min!"
        )
        
    deletion_mean_list = auc_stats["deletion"]["mean"]
    deletion_ci_list = auc_stats["deletion"]["ci"]
    if deletion_direction == "max":
        deletion_best_idx = np.argmax(deletion_mean_list)
    elif deletion_direction == "min":
        deletion_best_idx = np.argmin(deletion_mean_list)
    else:
        raise ValueError(
            f"deietion_direction={deietion_direction} should be max or min!"
        )
    
    text_list = []
    for i in range(len(insertion_mean_list)):
        insertion_mean = insertion_mean_list[i]
        insertion_ci = insertion_ci_list[i]
        if insertion_mean < 0.01:
            insertion_text = (
                "{:.2e}".format(insertion_mean)
                + " $\pm$ "
                + "{:.2e}".format(insertion_ci)
            )
        else:
            insertion_text = f"{insertion_mean:.3f} $\pm$ {insertion_ci:.3f}"
        if i == insertion_best_idx and bold_best:
            insertion_text = "\\textbf{" + insertion_text + "}"
            
        deletion_mean = deletion_mean_list[i]
        deletion_ci = deletion_ci_list[i]
        if deletion_mean < 0.01:
            deletion_text = (
                "{:.2e}".format(deletion_mean)
                + " $\pm$ "
                + "{:.2e}".format(deletion_ci)
            )
        else:
            deletion_text = f"{deletion_mean:.3f} $\pm$ {deletion_ci:.3f}"
        if i == deletion_best_idx and bold_best:
            deletion_text = "\\textbf{" + deletion_text + "}"
            
        text = insertion_text + " & " + deletion_text
        text_list.append(text)
    return text_list

In [9]:
def print_aucs(
    eval_name: str,
    normalize_similarity: bool,
    different_classes: bool,
    insertion_direction: str = "max",
    deletion_direction: str = "min",
):
    attribution_list = ["int_grad", "gradient_shap", "rise"]
    dataset_encoder_combos = [
        ("imagenet", "simclr_x1"),
        ("cifar", "simsiam_18"),
        ("mura", "classifier_18"),
    ]
    for attribution in attribution_list:
        print(attribution)
        print("-" * len(attribution))
        label_free_text = "Label-Free"
        contrastive_label_free_text = "Contrastive Label-Free"
        corpus_text = "Corpus"
        cocoa_text = "COCOA"
        for dataset_encoder in dataset_encoder_combos:
            text_list = get_formatted_aucs(
                insertion_direction=insertion_direction,
                deletion_direction=deletion_direction,
                dataset=dataset_encoder[0],
                encoder=dataset_encoder[1],
                attribution=attribution,
                eval_name=eval_name,
                normalize_similarity=normalize_similarity,
                different_classes=different_classes,
            )
            label_free_text += f" & {text_list[0]}"
            contrastive_label_free_text += f" & {text_list[1]}"
            corpus_text += f" & {text_list[2]}"
            cocoa_text += f" & {text_list[3]}"
        print("& " + label_free_text + " \\\\")
        print("& " + contrastive_label_free_text + " \\\\")
        print("& " + corpus_text + " \\\\")
        print("& " + cocoa_text + " \\\\")
        print("")

    print("random")
    print("------")
    random_text = "None"
    for dataset_encoder in dataset_encoder_combos:
        text_list = get_formatted_aucs(
            insertion_direction=insertion_direction,
            deletion_direction=deletion_direction,
            bold_best=False,
            dataset=dataset_encoder[0],
            encoder=dataset_encoder[1],
            attribution="random_baseline",
            eval_name=eval_name,
            normalize_similarity=True,  # Does not matter for random baseline.
            different_classes=different_classes,
        )
        random_text += f" & {text_list[0]}"
    print("& " + random_text + " \\\\")

## Corpus Majority Probability (Cosine Similarity & Same Class)

In [5]:
print_aucs(eval_name="corpus_majority_prob", normalize_similarity=True, different_classes=False)

int_grad
--------
& Label-Free & 0.362 $\pm$ 0.005 & 0.136 $\pm$ 0.004 & \textbf{0.403 $\pm$ 0.010} & 0.249 $\pm$ 0.007 & 0.631 $\pm$ 0.040 & 0.513 $\pm$ 0.027 \\
& Contrastive Label-Free & 0.377 $\pm$ 0.005 & 0.125 $\pm$ 0.003 & 0.401 $\pm$ 0.011 & 0.243 $\pm$ 0.007 & 0.690 $\pm$ 0.019 & 0.453 $\pm$ 0.025 \\
& Corpus & 0.377 $\pm$ 0.005 & 0.147 $\pm$ 0.002 & 0.355 $\pm$ 0.014 & 0.249 $\pm$ 0.010 & 0.653 $\pm$ 0.014 & 0.500 $\pm$ 0.039 \\
& COCOA & \textbf{0.422 $\pm$ 0.006} & \textbf{0.119 $\pm$ 0.003} & 0.386 $\pm$ 0.012 & \textbf{0.230 $\pm$ 0.011} & \textbf{0.807 $\pm$ 0.013} & \textbf{0.330 $\pm$ 0.030} \\

gradient_shap
-------------
& Label-Free & 0.409 $\pm$ 0.004 & 0.131 $\pm$ 0.001 & 0.500 $\pm$ 0.008 & 0.244 $\pm$ 0.013 & 0.691 $\pm$ 0.038 & 0.523 $\pm$ 0.033 \\
& Contrastive Label-Free & 0.411 $\pm$ 0.003 & 0.127 $\pm$ 0.002 & 0.500 $\pm$ 0.009 & 0.238 $\pm$ 0.012 & 0.697 $\pm$ 0.037 & 0.510 $\pm$ 0.018 \\
& Corpus & 0.421 $\pm$ 0.006 & 0.136 $\pm$ 0.001 & 0.478 $\pm$ 0.008

## Corpus Majority Probability (Cosine Similarity & Different Classes)

In [6]:
print_aucs(eval_name="corpus_majority_prob", normalize_similarity=True, different_classes=True)

int_grad
--------
& Label-Free & 3.53e-04 $\pm$ 1.06e-04 & 4.44e-04 $\pm$ 1.01e-04 & 0.061 $\pm$ 0.004 & 0.079 $\pm$ 0.004 & 0.394 $\pm$ 0.028 & 0.481 $\pm$ 0.031 \\
& Contrastive Label-Free & 3.36e-04 $\pm$ 1.13e-04 & 4.44e-04 $\pm$ 1.00e-04 & 0.062 $\pm$ 0.004 & 0.082 $\pm$ 0.004 & 0.354 $\pm$ 0.022 & 0.518 $\pm$ 0.027 \\
& Corpus & 1.09e-03 $\pm$ 2.85e-04 & 2.26e-04 $\pm$ 5.31e-05 & 0.094 $\pm$ 0.003 & 0.066 $\pm$ 0.003 & 0.609 $\pm$ 0.017 & 0.262 $\pm$ 0.029 \\
& COCOA & \textbf{1.69e-03 $\pm$ 5.07e-04} & \textbf{1.55e-04 $\pm$ 2.21e-05} & \textbf{0.099 $\pm$ 0.004} & \textbf{0.059 $\pm$ 0.005} & \textbf{0.647 $\pm$ 0.017} & \textbf{0.213 $\pm$ 0.030} \\

gradient_shap
-------------
& Label-Free & 2.05e-04 $\pm$ 5.96e-05 & 5.55e-04 $\pm$ 1.11e-04 & 0.054 $\pm$ 0.004 & 0.080 $\pm$ 0.004 & 0.362 $\pm$ 0.031 & 0.469 $\pm$ 0.021 \\
& Contrastive Label-Free & 2.02e-04 $\pm$ 5.06e-05 & 5.10e-04 $\pm$ 9.65e-05 & 0.053 $\pm$ 0.002 & 0.080 $\pm$ 0.004 & 0.361 $\pm$ 0.022 & 0.477 $\pm$ 0.018

## Explicand Probability (Cosine Similarity & Same Class)

In [7]:
print_aucs(eval_name="explicand_pred_prob", normalize_similarity=True, different_classes=False)

int_grad
--------
& Label-Free & 0.399 $\pm$ 0.007 & 0.143 $\pm$ 0.004 & \textbf{0.412 $\pm$ 0.008} & 0.255 $\pm$ 0.008 & 0.642 $\pm$ 0.043 & 0.491 $\pm$ 0.015 \\
& Contrastive Label-Free & 0.414 $\pm$ 0.006 & \textbf{0.131 $\pm$ 0.004} & 0.411 $\pm$ 0.011 & 0.250 $\pm$ 0.008 & \textbf{0.717 $\pm$ 0.023} & \textbf{0.416 $\pm$ 0.018} \\
& Corpus & 0.386 $\pm$ 0.006 & 0.164 $\pm$ 0.005 & 0.358 $\pm$ 0.013 & 0.258 $\pm$ 0.011 & 0.559 $\pm$ 0.029 & 0.595 $\pm$ 0.028 \\
& COCOA & \textbf{0.435 $\pm$ 0.006} & 0.132 $\pm$ 0.005 & 0.387 $\pm$ 0.012 & \textbf{0.241 $\pm$ 0.011} & 0.713 $\pm$ 0.032 & 0.425 $\pm$ 0.025 \\

gradient_shap
-------------
& Label-Free & 0.455 $\pm$ 0.004 & 0.138 $\pm$ 0.002 & 0.512 $\pm$ 0.008 & 0.249 $\pm$ 0.012 & \textbf{0.747 $\pm$ 0.029} & 0.478 $\pm$ 0.021 \\
& Contrastive Label-Free & 0.458 $\pm$ 0.004 & \textbf{0.135 $\pm$ 0.003} & \textbf{0.512 $\pm$ 0.010} & 0.243 $\pm$ 0.011 & 0.742 $\pm$ 0.035 & \textbf{0.460 $\pm$ 0.009} \\
& Corpus & 0.444 $\pm$ 0.006 & 0

## Explicand Representation Shift (Cosine Similarity & Same Class)

In [8]:
print_aucs(
    eval_name="explicand_rep_shift",
    normalize_similarity=True,
    different_classes=False,
    insertion_direction="min",
    deletion_direction="max",
)

int_grad
--------
& Label-Free & 8.819 $\pm$ 0.079 & 11.626 $\pm$ 0.097 & \textbf{0.125 $\pm$ 0.001} & 0.136 $\pm$ 0.001 & 10.752 $\pm$ 0.541 & 11.563 $\pm$ 0.667 \\
& Contrastive Label-Free & \textbf{8.719 $\pm$ 0.084} & \textbf{11.755 $\pm$ 0.097} & 0.125 $\pm$ 0.001 & \textbf{0.136 $\pm$ 0.001} & \textbf{10.025 $\pm$ 0.387} & \textbf{12.866 $\pm$ 0.442} \\
& Corpus & 9.659 $\pm$ 0.080 & 10.904 $\pm$ 0.098 & 0.131 $\pm$ 0.001 & 0.134 $\pm$ 0.001 & 12.224 $\pm$ 0.543 & 10.354 $\pm$ 0.721 \\
& COCOA & 9.234 $\pm$ 0.083 & 11.355 $\pm$ 0.096 & 0.130 $\pm$ 0.001 & 0.134 $\pm$ 0.001 & 10.920 $\pm$ 0.561 & 12.628 $\pm$ 0.548 \\

gradient_shap
-------------
& Label-Free & 7.936 $\pm$ 0.054 & 11.826 $\pm$ 0.107 & 0.120 $\pm$ 0.001 & \textbf{0.139 $\pm$ 0.001} & \textbf{7.589 $\pm$ 0.404} & \textbf{12.184 $\pm$ 0.904} \\
& Contrastive Label-Free & \textbf{7.912 $\pm$ 0.051} & \textbf{11.872 $\pm$ 0.111} & \textbf{0.120 $\pm$ 0.001} & 0.139 $\pm$ 0.001 & 8.092 $\pm$ 0.440 & 11.789 $\pm$ 0.917 \

## Explicand Representation Shift (Cosine Similarity & Different Classes)

In [9]:
print_aucs(
    eval_name="explicand_rep_shift",
    normalize_similarity=True,
    different_classes=True,
    insertion_direction="min",
    deletion_direction="max",
)

int_grad
--------
& Label-Free & 8.764 $\pm$ 0.141 & 11.556 $\pm$ 0.207 & \textbf{0.125 $\pm$ 0.001} & 0.136 $\pm$ 0.000 & 11.025 $\pm$ 0.366 & 11.727 $\pm$ 0.781 \\
& Contrastive Label-Free & \textbf{8.664 $\pm$ 0.137} & \textbf{11.676 $\pm$ 0.209} & 0.125 $\pm$ 0.001 & \textbf{0.136 $\pm$ 0.000} & \textbf{10.096 $\pm$ 0.352} & \textbf{13.011 $\pm$ 0.607} \\
& Corpus & 10.956 $\pm$ 0.214 & 9.584 $\pm$ 0.131 & 0.134 $\pm$ 0.000 & 0.132 $\pm$ 0.000 & 13.479 $\pm$ 0.579 & 9.988 $\pm$ 0.378 \\
& COCOA & 10.048 $\pm$ 0.179 & 10.573 $\pm$ 0.174 & 0.133 $\pm$ 0.000 & 0.132 $\pm$ 0.000 & 12.391 $\pm$ 0.650 & 10.988 $\pm$ 0.475 \\

gradient_shap
-------------
& Label-Free & 7.857 $\pm$ 0.102 & 11.754 $\pm$ 0.194 & 0.121 $\pm$ 0.001 & \textbf{0.140 $\pm$ 0.001} & \textbf{7.650 $\pm$ 0.346} & \textbf{12.353 $\pm$ 0.667} \\
& Contrastive Label-Free & \textbf{7.828 $\pm$ 0.099} & \textbf{11.802 $\pm$ 0.195} & \textbf{0.121 $\pm$ 0.001} & 0.140 $\pm$ 0.001 & 8.057 $\pm$ 0.475 & 12.226 $\pm$ 0.780 \

## Corpus Majority Probability (Dot Product & Same Class)

In [12]:
print_aucs(eval_name="corpus_majority_prob", normalize_similarity=False, different_classes=False)

int_grad
--------
& Label-Free & 0.364 $\pm$ 0.004 & 0.125 $\pm$ 0.003 & 0.343 $\pm$ 0.014 & 0.237 $\pm$ 0.012 & 0.696 $\pm$ 0.020 & 0.451 $\pm$ 0.018 \\
& Contrastive Label-Free & 0.372 $\pm$ 0.005 & 0.119 $\pm$ 0.003 & 0.354 $\pm$ 0.017 & 0.234 $\pm$ 0.012 & 0.690 $\pm$ 0.022 & 0.452 $\pm$ 0.029 \\
& Corpus & 0.384 $\pm$ 0.004 & 0.126 $\pm$ 0.002 & 0.346 $\pm$ 0.012 & 0.240 $\pm$ 0.010 & 0.767 $\pm$ 0.010 & 0.392 $\pm$ 0.018 \\
& COCOA & \textbf{0.415 $\pm$ 0.006} & \textbf{0.113 $\pm$ 0.002} & \textbf{0.379 $\pm$ 0.012} & \textbf{0.222 $\pm$ 0.009} & \textbf{0.806 $\pm$ 0.012} & \textbf{0.325 $\pm$ 0.031} \\

gradient_shap
-------------
& Label-Free & 0.408 $\pm$ 0.004 & 0.130 $\pm$ 0.002 & 0.475 $\pm$ 0.011 & 0.235 $\pm$ 0.013 & 0.699 $\pm$ 0.040 & 0.515 $\pm$ 0.018 \\
& Contrastive Label-Free & 0.413 $\pm$ 0.003 & 0.126 $\pm$ 0.002 & 0.487 $\pm$ 0.011 & 0.229 $\pm$ 0.012 & 0.700 $\pm$ 0.041 & 0.510 $\pm$ 0.019 \\
& Corpus & 0.423 $\pm$ 0.004 & 0.130 $\pm$ 0.002 & 0.444 $\pm$ 0.011

## Corpus Majority Probability (Dot Product & Different Classes)

In [13]:
print_aucs(eval_name="corpus_majority_prob", normalize_similarity=False, different_classes=True)

int_grad
--------
& Label-Free & 2.74e-04 $\pm$ 9.69e-05 & 5.42e-04 $\pm$ 1.12e-04 & 0.066 $\pm$ 0.003 & 0.078 $\pm$ 0.004 & 0.352 $\pm$ 0.024 & 0.522 $\pm$ 0.029 \\
& Contrastive Label-Free & 2.75e-04 $\pm$ 1.00e-04 & 5.00e-04 $\pm$ 1.07e-04 & 0.064 $\pm$ 0.004 & 0.078 $\pm$ 0.004 & 0.347 $\pm$ 0.024 & 0.529 $\pm$ 0.023 \\
& Corpus & 5.60e-04 $\pm$ 2.54e-04 & 4.13e-04 $\pm$ 8.00e-05 & 0.079 $\pm$ 0.004 & 0.070 $\pm$ 0.004 & 0.529 $\pm$ 0.018 & 0.381 $\pm$ 0.026 \\
& COCOA & \textbf{1.58e-03 $\pm$ 4.99e-04} & \textbf{1.64e-04 $\pm$ 2.97e-05} & \textbf{0.100 $\pm$ 0.005} & \textbf{0.060 $\pm$ 0.005} & \textbf{0.657 $\pm$ 0.017} & \textbf{0.210 $\pm$ 0.024} \\

gradient_shap
-------------
& Label-Free & 1.88e-04 $\pm$ 4.25e-05 & 6.01e-04 $\pm$ 1.13e-04 & 0.055 $\pm$ 0.005 & 0.082 $\pm$ 0.006 & 0.343 $\pm$ 0.016 & 0.488 $\pm$ 0.020 \\
& Contrastive Label-Free & 2.03e-04 $\pm$ 3.73e-05 & 5.30e-04 $\pm$ 7.78e-05 & 0.053 $\pm$ 0.004 & 0.082 $\pm$ 0.005 & 0.353 $\pm$ 0.019 & 0.478 $\pm$ 0.019

## Explicand Probability (Dot Product & Same Class)

In [14]:
print_aucs(eval_name="explicand_pred_prob", normalize_similarity=False, different_classes=False)

int_grad
--------
& Label-Free & 0.404 $\pm$ 0.007 & 0.131 $\pm$ 0.003 & 0.353 $\pm$ 0.014 & 0.243 $\pm$ 0.013 & \textbf{0.764 $\pm$ 0.021} & \textbf{0.379 $\pm$ 0.020} \\
& Contrastive Label-Free & 0.414 $\pm$ 0.007 & 0.124 $\pm$ 0.003 & 0.362 $\pm$ 0.017 & 0.240 $\pm$ 0.013 & 0.732 $\pm$ 0.029 & 0.401 $\pm$ 0.019 \\
& Corpus & 0.411 $\pm$ 0.007 & 0.135 $\pm$ 0.003 & 0.351 $\pm$ 0.012 & 0.248 $\pm$ 0.011 & 0.712 $\pm$ 0.035 & 0.425 $\pm$ 0.017 \\
& COCOA & \textbf{0.432 $\pm$ 0.007} & \textbf{0.124 $\pm$ 0.004} & \textbf{0.380 $\pm$ 0.012} & \textbf{0.233 $\pm$ 0.010} & 0.711 $\pm$ 0.034 & 0.421 $\pm$ 0.026 \\

gradient_shap
-------------
& Label-Free & 0.454 $\pm$ 0.006 & 0.137 $\pm$ 0.003 & 0.484 $\pm$ 0.010 & 0.238 $\pm$ 0.012 & \textbf{0.761 $\pm$ 0.037} & 0.465 $\pm$ 0.019 \\
& Contrastive Label-Free & 0.461 $\pm$ 0.004 & \textbf{0.133 $\pm$ 0.003} & 0.498 $\pm$ 0.012 & 0.233 $\pm$ 0.011 & 0.744 $\pm$ 0.036 & \textbf{0.460 $\pm$ 0.011} \\
& Corpus & 0.455 $\pm$ 0.006 & 0.141 $\pm

## Explicand Representation Shift (Dot Product & Same Class)

In [15]:
print_aucs(
    eval_name="explicand_rep_shift",
    normalize_similarity=False,
    different_classes=False,
    insertion_direction="min",
    deletion_direction="max",
)

int_grad
--------
& Label-Free & 9.580 $\pm$ 0.074 & 11.325 $\pm$ 0.103 & 0.128 $\pm$ 0.001 & 0.135 $\pm$ 0.001 & 10.216 $\pm$ 0.277 & 12.992 $\pm$ 0.536 \\
& Contrastive Label-Free & \textbf{9.372 $\pm$ 0.082} & \textbf{11.500 $\pm$ 0.099} & \textbf{0.128 $\pm$ 0.001} & \textbf{0.136 $\pm$ 0.001} & \textbf{10.189 $\pm$ 0.440} & \textbf{13.018 $\pm$ 0.384} \\
& Corpus & 9.817 $\pm$ 0.076 & 11.045 $\pm$ 0.103 & 0.130 $\pm$ 0.001 & 0.134 $\pm$ 0.001 & 10.665 $\pm$ 0.343 & 12.774 $\pm$ 0.593 \\
& COCOA & 9.493 $\pm$ 0.085 & 11.306 $\pm$ 0.099 & 0.129 $\pm$ 0.001 & 0.134 $\pm$ 0.001 & 11.046 $\pm$ 0.590 & 12.777 $\pm$ 0.581 \\

gradient_shap
-------------
& Label-Free & 8.188 $\pm$ 0.053 & 11.686 $\pm$ 0.108 & 0.122 $\pm$ 0.001 & 0.137 $\pm$ 0.001 & \textbf{7.948 $\pm$ 0.369} & 11.620 $\pm$ 0.964 \\
& Contrastive Label-Free & \textbf{8.030 $\pm$ 0.052} & \textbf{11.806 $\pm$ 0.112} & \textbf{0.121 $\pm$ 0.001} & \textbf{0.138 $\pm$ 0.001} & 8.056 $\pm$ 0.404 & \textbf{11.707 $\pm$ 0.869} \